In [2]:
#imports
import optuna
from model import *

In [3]:
## Optuna Function
def objective(trial):
    # Detect device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Suggest hyperparameters
    # Restrict d_model and nhead to best from first optuna to get better insights (no inf)
    d_model = 48
    nhead = 6
    
    seasons = trial.suggest_int("seasons", 0, 1)
    days = trial.suggest_int("weekdays", 0, 1)
    weekday_weekend = trial.suggest_int("weekday_weekend", 0, 1)
    months = trial.suggest_int("months", 0, 1)
    min_demand = trial.suggest_int("min_demand", 0, 1)
    avg_demand = trial.suggest_int("avg_demand", 0, 1)
    max_demand = trial.suggest_int("max_demand", 0, 1)
    avg_temp = trial.suggest_int("avg_temp", 0, 1)
    max_temp = trial.suggest_int("max_temp", 0, 1)
    min_temp = trial.suggest_int("min_temp", 0, 1)
    hd = trial.suggest_int("hd", 0, 1)
    cd = trial.suggest_int("cd", 0, 1)
    features = ['sum_30_min_demand']  # Always include sum_30_min_demand
    cases = [seasons, days, weekday_weekend, months, min_demand, avg_demand, max_demand, avg_temp, max_temp, min_temp, hd, cd]
    feature_names = [['is_summer', 'is_autumn', 'is_winter', 'is_spring'], 
                     ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday'], 
                     ['is_weekday', 'is_weekend'], 
                     ['is_jan', 'is_feb', 'is_mar', 'is_apr', 'is_may', 'is_jun', 'is_jul', 'is_aug', 'is_sep', 'is_oct', 'is_nov', 'is_dec'], 
                     ['min_30_min_demand'], 
                     ['avg_30_min_demand'], 
                     ['max_30_min_demand'], 
                     ['avg_temp'], 
                     ['max_temp'], 
                     ['min_temp'], 
                     ['hd_next_24h'], 
                     ['cd_next_24h']]
    for i in range(len(cases)):
        if cases[i] == 1:
            features += feature_names[i]

    params = {
        'learning_rate': 0.002,
        'batch_size': 16,
        'seq_length': 14,
        'transformer_encoder_layer_params': {
            'd_model': d_model,
            'nhead': nhead,
            'dim_feedforward': 276,
            'dropout': 0.0579,
            'activation': 'relu'
        },
        'transformer_layer_params': {
            'num_layers': 3,
        },
        'dataset': '2010-2019',
        'train_test_split': '80:20',
        'features': features
    }

    input_dim = len(params['features'])

    sequences, targets, datetimes, scaler_X = prepare_data(params)

    # Build and train model
    model, train_losses, val_losses = train_transformer_model(sequences, targets, input_dim, datetimes, params)

    # 5. Evaluate and plot results
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    test_mask = pd.to_datetime(datetimes).dt.year == 2019
    seq_mask = test_mask[params['seq_length']:].to_numpy()
    predictions = evaluate_model(model, sequences[seq_mask], targets[seq_mask], device)

    # Inverse transform predictions to original scale
    # Create dummy array for inverse transform
    dummy = np.zeros((len(predictions), input_dim))
    dummy[:, 0] = predictions.flatten()
    predictions_original = scaler_X.inverse_transform(dummy)[:, 0]

    dummy[:, 0] = targets[seq_mask].numpy().flatten()
    targets_original = scaler_X.inverse_transform(dummy)[:, 0]

    # Calculate mae
    mae = np.mean(np.abs(predictions_original - targets_original))
    return mae

In [4]:
study = optuna.create_study(storage="sqlite:///db.sqlite3", study_name="feature_selection", load_if_exists=True)
study.optimize(objective, n_trials=100, n_jobs=1)

study.best_params 

[I 2025-09-27 11:11:23,833] Using an existing study with name 'feature_selection' instead of creating a new one.


Using device: cuda


[I 2025-09-27 11:12:53,465] Trial 47 finished with value: 11981.02781453678 and parameters: {'seasons': 1, 'weekdays': 0, 'weekday_weekend': 1, 'months': 0, 'min_demand': 1, 'avg_demand': 1, 'max_demand': 1, 'avg_temp': 1, 'max_temp': 1, 'min_temp': 0, 'hd': 0, 'cd': 0}. Best is trial 37 with value: 11393.430537639542.


Using device: cuda


[I 2025-09-27 11:14:29,693] Trial 48 finished with value: 12083.044497896271 and parameters: {'seasons': 1, 'weekdays': 0, 'weekday_weekend': 1, 'months': 0, 'min_demand': 1, 'avg_demand': 1, 'max_demand': 1, 'avg_temp': 1, 'max_temp': 1, 'min_temp': 0, 'hd': 0, 'cd': 0}. Best is trial 37 with value: 11393.430537639542.


...

Using device: cuda


[I 2025-09-27 12:23:17,798] Trial 125 finished with value: 11668.26364031109 and parameters: {'seasons': 1, 'weekdays': 0, 'weekday_weekend': 1, 'months': 1, 'min_demand': 0, 'avg_demand': 0, 'max_demand': 1, 'avg_temp': 0, 'max_temp': 1, 'min_temp': 0, 'hd': 0, 'cd': 1}. Best is trial 51 with value: 11000.782831201763.


Using device: cuda


[W 2025-09-27 12:24:03,491] Trial 126 failed with parameters: {'seasons': 1, 'weekdays': 0, 'weekday_weekend': 1, 'months': 1, 'min_demand': 0, 'avg_demand': 0, 'max_demand': 1, 'avg_temp': 0, 'max_temp': 1, 'min_temp': 0, 'hd': 0, 'cd': 1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/harry/personal/uni/project/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_25009/720489053.py", line 65, in objective
    model, train_losses, val_losses = train_transformer_model(sequences, targets, input_dim, datetimes, params)
  File "/home/harry/personal/uni/project/individual/harry/model/model.py", line 134, in train_transformer_model
    predictions = model(batch_X)
  File "/home/harry/personal/uni/project/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  Fi

KeyboardInterrupt: 

In [1]:
study.best_params

NameError: name 'study' is not defined